In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
243,AFG,Asia,Afghanistan,2020-08-30,38143.0,3.0,20.571,1402.0,0.0,2.143,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
418,ALB,Europe,Albania,2020-08-30,9279.0,84.0,143.429,275.0,4.0,4.286,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
662,DZA,Africa,Algeria,2020-08-30,43782.0,379.0,387.714,1491.0,8.0,9.571,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-29,France,271874.0,0.0,30642.0,0.0,0.0,0.0
0,2020-08-29,China,89866.0,0.0,4724.0,0.0,0.0,0.0
0,2020-08-29,Italy,266664.0,0.0,35497.0,0.0,0.0,0.0
0,2020-08-29,Spain,447632.0,0.0,29039.0,0.0,0.0,0.0
0,2020-08-29,United States,5998150.0,0.0,182991.0,0.0,0.0,0.0
0,2020-08-29,World,25219820.0,0.0,845695.0,0.0,0.0,0.0
0,2020-08-29,United Kingdom,333176.0,0.0,41532.0,0.0,0.0,0.0
0,2020-08-29,Germany,242389.0,0.0,9298.0,0.0,0.0,0.0
0,2020-08-29,Iran,373427.0,0.0,21507.0,0.0,0.0,0.0
0,2020-08-29,Turkey,266755.0,0.0,6266.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-30,France,271874.0,272530.0,30642.0,30602.0,-656.0,40.0
0,2020-08-30,China,89866.0,89863.0,4724.0,4721.0,3.0,3.0
0,2020-08-30,Italy,266664.0,266853.0,35497.0,35473.0,-189.0,24.0
0,2020-08-29,Spain,447632.0,439286.0,29039.0,29011.0,8346.0,28.0
0,2020-08-30,United States,5998150.0,5961582.0,182991.0,182779.0,36568.0,212.0
0,2020-08-30,World,25219820.0,25029408.0,845695.0,843158.0,190412.0,2537.0
0,2020-08-30,United Kingdom,333176.0,332752.0,41532.0,41498.0,424.0,34.0
0,2020-08-30,Germany,242389.0,241771.0,9298.0,9295.0,618.0,3.0
0,2020-08-30,Iran,373427.0,371816.0,21507.0,21359.0,1611.0,148.0
0,2020-08-30,Turkey,266755.0,267064.0,6266.0,6284.0,-309.0,-18.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")